# Knet CNN Example

In [1]:
# After installing and starting Julia run the following to install the required packages:
# Pkg.init(); Pkg.update()
# for p in ("CUDAdrv","IJulia","Knet"); Pkg.add(p); end
# Pkg.checkout("Knet","ilkarman") # make sure we have the right Knet version
# Pkg.build("Knet")

In [2]:
using Knet; @show gpu()
True=true # so we can read the python params
include("common/params.py");

┌ Info: Recompiling stale cache file /kuacc/users/dyuret/.julia/compiled/v1.0/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1184


gpu() = 0


In [3]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
#println("Knet: ", Pkg.installed("Knet"))
println("GPU: ", read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String))

OS: Linux
Julia: 1.0.0
GPU: Tesla K80



In [4]:
# define model
function initmodel(; atype=KnetArray, dtype=Float32, winit=xavier, binit=zeros)
    w(dims...)=atype(winit(dtype,dims...))
    b(dims...)=atype(binit(dtype,dims...))
    return Any[
        w(3,3,3,50), b(1,1,50,1),
        w(3,3,50,50), b(1,1,50,1),
        w(3,3,50,100), b(1,1,100,1),
        w(3,3,100,100), b(1,1,100,1),
        w(512,6400), b(512,1),
        w(10,512), b(10,1)
    ]
end;

In [5]:
# define loss and its gradient
function predict(w,x)
    convbias(x,w,b) = conv4(w,x;padding=1) .+ b
    fc(x,w,b) = w * mat(x) .+ b;
    x = relu.(convbias(x,w[1],w[2]))
    x = relu.(pool(convbias(x,w[3],w[4])))
    x = dropout(x,0.25)
    x = relu.(convbias(x,w[5],w[6]))
    x = relu.(pool(convbias(x,w[7],w[8])))
    x = dropout(x,0.25)
    x = relu.(fc(x,w[9],w[10]))
    x = dropout(x,0.5)
    return fc(x,w[11],w[12])
end

loss(w,x,y)=nll(predict(w,x),y) # nll: negative log likelihood
lossgradient = grad(loss);

In [6]:
# load data
include(Knet.dir("data","cifar.jl"))
@time (xtrn,ytrn,xtst,ytst,lbls)=cifar10()
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

┌ Info: Reading cifar-10-binary.tar.gz...
└ @ Main /kuacc/users/dyuret/.julia/dev/Knet/data/cifar.jl:41


  3.030743 seconds (2.94 M allocations: 1.859 GiB, 22.35% gc time)
32×32×3×50000 Array{Float32,4}
50000-element Array{UInt8,1}
32×32×3×10000 Array{Float32,4}
10000-element Array{UInt8,1}


In [7]:
# prepare for training
model = optim = nothing; Knet.gc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [8]:
# cold start
@time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true,xtype=KnetArray)
    grads = lossgradient(model, x, y)
    update!(model, grads, optim)
end

 26.533408 seconds (23.48 M allocations: 1.719 GiB, 6.96% gc time)


In [9]:
# prepare for training
model = optim = nothing; Knet.gc() # Clear memory from last run
model = initmodel()
optim = optimizers(model, Momentum; lr=LR, gamma=MOMENTUM);

In [10]:
# 159s
@info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true,xtype=KnetArray)
        grads = lossgradient(model, x, y)
        update!(model, grads, optim)
    end
end

┌ Info: Training...
└ @ Main In[10]:2


 14.097721 seconds (1.76 M allocations: 661.696 MiB, 0.39% gc time)
 13.765463 seconds (1.76 M allocations: 661.564 MiB, 0.45% gc time)
 13.776932 seconds (1.76 M allocations: 661.564 MiB, 0.39% gc time)
 13.786659 seconds (1.76 M allocations: 661.564 MiB, 0.39% gc time)
 13.795239 seconds (1.76 M allocations: 661.564 MiB, 0.43% gc time)
 13.788541 seconds (1.76 M allocations: 661.564 MiB, 0.41% gc time)
 13.786247 seconds (1.76 M allocations: 661.564 MiB, 0.39% gc time)
 13.792936 seconds (1.76 M allocations: 661.564 MiB, 0.38% gc time)
 13.815097 seconds (1.76 M allocations: 661.564 MiB, 0.44% gc time)
 13.790948 seconds (1.76 M allocations: 661.564 MiB, 0.42% gc time)
138.201485 seconds (17.61 M allocations: 6.461 GiB, 0.41% gc time)


In [11]:
# test accuracy 77.54
testdata = minibatch(xtst,ytst,BATCHSIZE;xtype=KnetArray)
@time accuracy(model,testdata,predict)

  1.703879 seconds (1.60 M allocations: 195.873 MiB, 2.31% gc time)


0.7270633012820513